In [ ]:
# Demo de Integración Completa - Sistema de Gestión de Ventas
## Avance 2 y 3: Patrones de Diseño + SQL Avanzado

Este notebook integra y demuestra todas las funcionalidades del sistema de gestión de ventas para una cadena de supermercados, implementando:

## 🎯 **Avance 2: Patrones de Diseño**
1. **Singleton Pattern** - Conexión única a la base de datos
2. **Factory Method Pattern** - Creación de modelos de datos
3. **Builder Pattern** - Construcción fluida de consultas SQL
4. **Decorator Pattern** - Funcionalidades adicionales (logging, timing, caching)
5. **Strategy Pattern** - Diferentes algoritmos de análisis

## 🚀 **Avance 3: SQL Avanzado**
6. **Common Table Expressions (CTEs)** - Consultas jerárquicas y complejas
7. **Window Functions** - Análisis avanzado con ROW_NUMBER(), RANK(), etc.
8. **Objetos SQL Avanzados** - Funciones, Procedimientos, Triggers, Vistas, Índices
9. **Optimización de Performance** - Índices estratégicos y análisis de planes de ejecución
10. **Automatización de Procesos** - Triggers y procedimientos para operaciones en tiempo real

---

### 📊 **Objetivo del Sistema**
Desarrollar un sistema eficiente capaz de manejar grandes volúmenes de datos y generar reportes en tiempo real para la toma de decisiones estratégicas en la cadena de supermercados.


In [ ]:
# Importaciones necesarias
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Importar clases del Avance 2 (Patrones de Diseño)
from src.database.connection import DatabaseConnection
from src.database.query_examples import QueryExamples
from src.utils.model_factory import ModelFactoryRegistry, CategoryFactory, ProductFactory
from src.utils.query_builder import SQLQueryBuilder, QueryBuilderDirector
from src.utils.decorators import database_operation, timing_decorator, logging_decorator
from src.utils.analysis_strategies import (
    RevenueAnalysisStrategy, 
    QuantityAnalysisStrategy, 
    CustomerBehaviorAnalysisStrategy,
    SalesAnalysisContext,
    AnalysisStrategyFactory
)

# Importar clases del Avance 3 (SQL Avanzado)
from src.database.advanced_queries import AdvancedSQLQueries
from src.database.sql_objects_demo import SQLObjectsDemo

print("✅ Todas las importaciones completadas exitosamente")
print("📦 Módulos cargados:")
print("   - Patrones de Diseño (Avance 2)")
print("   - SQL Avanzado y CTEs (Avance 3)")
print("   - Objetos SQL Avanzados (Avance 3)")


In [ ]:
# 1. Demostración del Singleton Pattern
print("🔗 SINGLETON PATTERN - Conexión a la Base de Datos")
print("=" * 50)

# Crear múltiples instancias y verificar que son la misma
db1 = DatabaseConnection()
db2 = DatabaseConnection()

print(f"Instancia 1 ID: {id(db1)}")
print(f"Instancia 2 ID: {id(db2)}")
print(f"¿Son la misma instancia? {db1 is db2}")

# Probar la conexión
try:
    connection_status = db1.test_connection()
    print(f"\n✅ Estado de conexión: {connection_status}")
    
    # Obtener información de tablas
    tables_info = db1.get_table_info()
    print(f"📊 Tablas disponibles: {len(tables_info)} encontradas")
    for table in tables_info[:5]:  # Mostrar primeras 5 tablas
        print(f"   - {table}")
        
except Exception as e:
    print(f"❌ Error de conexión: {e}")
    print("💡 Asegúrate de que la base de datos esté funcionando y el archivo .env configurado")


In [ ]:
# 2. Demostración del Factory Method Pattern
print("\n🏭 FACTORY METHOD PATTERN - Creación de Modelos")
print("=" * 50)

# Datos de ejemplo para crear modelos
category_data = {
    'category_id': 1,
    'category_name': 'Orgánicos',
    'description': 'Productos orgánicos frescos'
}

product_data = {
    'product_id': 101,
    'product_name': 'Manzanas Orgánicas',
    'price': 3.50,
    'category_id': 1,
    'class_type': 'Premium',
    'modify_date': datetime.now(),
    'resistant': True,
    'is_allergic': False,
    'vitality_days': 7
}

# Usar Factory Registry para crear objetos
registry = ModelFactoryRegistry()

# Crear Category usando Factory
category = registry.create_model('category', category_data)
print(f"✅ Categoría creada: {category.category_name}")
print(f"   ID: {category.category_id}")
print(f"   Descripción: {category.description}")

# Crear Product usando Factory
product = registry.create_model('product', product_data)
print(f"\n✅ Producto creado: {product.product_name}")
print(f"   ID: {product.product_id}")
print(f"   Precio: ${product.price}")
print(f"   Categoría: {product.category_id}")
print(f"   Clase: {product.class_type}")

# Demostrar extensibilidad - agregar nueva factory
print(f"\n📦 Factories registradas: {list(registry._factories.keys())}")


In [ ]:
# 3. Demostración del Builder Pattern
print("\n🔧 BUILDER PATTERN - Construcción de Consultas SQL")
print("=" * 50)

# Crear instancia del builder
builder = SQLQueryBuilder()
director = QueryBuilderDirector(builder)

# Construcción manual fluida
query1 = (builder
    .select("p.product_name", "p.price", "c.category_name")
    .from_table("products p")
    .join("categories c", "p.category_id = c.category_id")
    .where("p.price > 10")
    .order_by("p.price DESC")
    .limit(5)
    .build())

print("✅ Consulta construida manualmente:")
print(f"   {query1}")

# Usar el Director para patrones comunes
builder.reset()
query2 = director.build_sales_analysis_query()
print(f"\n✅ Consulta de análisis de ventas (Director):")
print(f"   {query2}")

# Consulta compleja con múltiples JOINs
builder.reset()
query3 = (builder
    .select("s.sale_date", "p.product_name", "c.category_name", 
            "s.quantity", "s.total_price")
    .from_table("sales s")
    .join("products p", "s.product_id = p.product_id")
    .join("categories c", "p.category_id = c.category_id")
    .where("s.sale_date >= CURDATE() - INTERVAL 30 DAY")
    .group_by("DATE(s.sale_date)", "c.category_name")
    .having("SUM(s.total_price) > 1000")
    .order_by("s.sale_date DESC")
    .build())

print(f"\n✅ Consulta compleja con múltiples JOINs:")
print(f"   {query3}")

print(f"\n📊 Ventajas del Builder Pattern:")
print("   - Construcción paso a paso legible")
print("   - Reutilización de componentes")
print("   - Validación automática de consultas")
print("   - Flexibilidad en la construcción")


In [ ]:
# 4. Demostración del Decorator Pattern
print("\n🎭 DECORATOR PATTERN - Funcionalidades Transversales")
print("=" * 50)

# Función de ejemplo para decorar
@timing_decorator
@logging_decorator
def sample_database_query():
    """Función de ejemplo que simula una consulta a la base de datos."""
    import time
    time.sleep(0.1)  # Simular tiempo de consulta
    return "Resultados de la consulta"

# Ejecutar función decorada
print("✅ Ejecutando función con decoradores múltiples:")
result = sample_database_query()
print(f"   Resultado: {result}")

# Demostrar el decorador de operaciones de base de datos
@database_operation
def get_product_count():
    """Obtiene el conteo de productos."""
    db = DatabaseConnection()
    query = "SELECT COUNT(*) as count FROM products"
    try:
        result = db.execute_query(query)
        return result[0]['count'] if result else 0
    except:
        return 0

print(f"\n✅ Usando decorador de operaciones de BD:")
try:
    count = get_product_count()
    print(f"   Total de productos: {count}")
except Exception as e:
    print(f"   Error capturado por decorador: {e}")

print(f"\n🔧 Decoradores implementados:")
print("   - timing_decorator: Mide tiempo de ejecución")
print("   - logging_decorator: Registra llamadas a métodos")
print("   - caching_decorator: Cache LRU para resultados")
print("   - error_handling_decorator: Manejo robusto de errores")
print("   - retry_decorator: Reintentos automáticos")
print("   - database_operation: Decorador compuesto")


In [ ]:
# 5. Demostración del Strategy Pattern
print("\n🎯 STRATEGY PATTERN - Algoritmos de Análisis")
print("=" * 50)

# Crear contexto de análisis
context = SalesAnalysisContext()

# Factory para crear estrategias
factory = AnalysisStrategyFactory()

# Datos de ejemplo para análisis
sales_data = [
    {'product_id': 1, 'quantity': 10, 'total_price': 100.0, 'customer_id': 1},
    {'product_id': 2, 'quantity': 5, 'total_price': 75.0, 'customer_id': 2},
    {'product_id': 1, 'quantity': 8, 'total_price': 80.0, 'customer_id': 1},
    {'product_id': 3, 'quantity': 15, 'total_price': 150.0, 'customer_id': 3},
]

print("📊 Datos de ventas de ejemplo:")
for i, sale in enumerate(sales_data, 1):
    print(f"   {i}. Producto {sale['product_id']}: ${sale['total_price']:.2f}")

# Estrategia 1: Análisis de Revenue
print("\n✅ Estrategia 1: Análisis de Revenue")
revenue_strategy = factory.create_strategy('revenue')
context.set_strategy(revenue_strategy)
revenue_result = context.execute_analysis(sales_data)
print(f"   Total Revenue: ${revenue_result:.2f}")

# Estrategia 2: Análisis de Quantity
print("\n✅ Estrategia 2: Análisis de Quantity")
quantity_strategy = factory.create_strategy('quantity')
context.set_strategy(quantity_strategy)
quantity_result = context.execute_analysis(sales_data)
print(f"   Total Quantity: {quantity_result} unidades")

# Estrategia 3: Análisis de Customer Behavior
print("\n✅ Estrategia 3: Análisis de Customer Behavior")
customer_strategy = factory.create_strategy('customer_behavior')
context.set_strategy(customer_strategy)
customer_result = context.execute_analysis(sales_data)
print(f"   Customers únicos: {customer_result}")

print(f"\n🔄 Estrategias disponibles: {factory.get_available_strategies()}")
print(f"📈 Ventajas del Strategy Pattern:")
print("   - Algoritmos intercambiables en tiempo de ejecución")
print("   - Fácil extensión con nuevas estrategias")
print("   - Separación clara de responsabilidades")
print("   - Cumple principio Abierto/Cerrado")


In [ ]:
# 6. Demostración de CTEs y Window Functions
print("\n📊 CONSULTAS AVANZADAS - CTEs y Window Functions")
print("=" * 60)

# Crear instancia de consultas avanzadas
advanced_queries = AdvancedSQLQueries()

try:
    # 1. Análisis de Performance de Ventas con CTEs
    print("\n✅ 1. Análisis de Performance de Ventas (CTEs + Window Functions)")
    print("-" * 50)
    
    sales_analysis = advanced_queries.sales_performance_analysis_with_cte()
    if isinstance(sales_analysis, pd.DataFrame) and not sales_analysis.empty:
        print(f"📈 Registros analizados: {len(sales_analysis)}")
        print("🏆 Top 3 vendedores por performance:")
        for i, row in sales_analysis.head(3).iterrows():
            print(f"   {i+1}. Vendedor {row.get('sales_person_id', 'N/A')}: "
                  f"${row.get('total_sales', 0):,.2f} "
                  f"(Ranking: {row.get('sales_rank', 'N/A')})")
    else:
        print("   No se encontraron datos de ventas")
    
    # 2. Segmentación de Clientes con RFM
    print("\n✅ 2. Segmentación de Clientes - Análisis RFM")
    print("-" * 50)
    
    customer_segmentation = advanced_queries.customer_segmentation_with_window_functions()
    if isinstance(customer_segmentation, pd.DataFrame) and not customer_segmentation.empty:
        print(f"👥 Clientes analizados: {len(customer_segmentation)}")
        print("🎯 Segmentación RFM (Recency, Frequency, Monetary):")
        
        # Mostrar distribución de segmentos
        if 'rfm_segment' in customer_segmentation.columns:
            segments = customer_segmentation['rfm_segment'].value_counts()
            for segment, count in segments.head(5).items():
                print(f"   - {segment}: {count} clientes")
        else:
            print("   Primeros 3 clientes:")
            for i, row in customer_segmentation.head(3).iterrows():
                print(f"   Cliente {row.get('customer_id', 'N/A')}: "
                      f"Recency={row.get('recency_days', 'N/A')}, "
                      f"Frequency={row.get('frequency', 'N/A')}, "
                      f"Monetary=${row.get('monetary', 0):.2f}")
    else:
        print("   No se encontraron datos de clientes")
    
    # 3. Tendencias de Productos
    print("\n✅ 3. Análisis de Tendencias de Productos")
    print("-" * 50)
    
    product_trends = advanced_queries.product_performance_trends_with_cte()
    if isinstance(product_trends, pd.DataFrame) and not product_trends.empty:
        print(f"📦 Productos analizados: {len(product_trends)}")
        print("📈 Top 3 productos con mejor tendencia:")
        for i, row in product_trends.head(3).iterrows():
            print(f"   {i+1}. Producto {row.get('product_id', 'N/A')}: "
                  f"${row.get('monthly_revenue', 0):,.2f} este mes "
                  f"(Crecimiento: {row.get('growth_rate', 0):.1f}%)")
    else:
        print("   No se encontraron datos de productos")
        
except Exception as e:
    print(f"❌ Error ejecutando consultas avanzadas: {e}")
    print("💡 Asegúrate de que la base de datos tenga datos y las tablas existan")

print(f"\n🔧 Técnicas utilizadas:")
print("   - Common Table Expressions (CTEs) para consultas jerárquicas")
print("   - ROW_NUMBER(), RANK(), DENSE_RANK() para ranking")
print("   - LAG(), LEAD() para análisis temporal")
print("   - Funciones de agregación con PARTITION BY")
print("   - Subconsultas correlacionadas optimizadas")


In [ ]:
# 7. Demostración de Objetos SQL Avanzados
print("\n🛠️  OBJETOS SQL AVANZADOS - Automatización y Optimización")
print("=" * 60)

# Crear instancia de demostración de objetos SQL
sql_demo = SQLObjectsDemo()

try:
    # 1. Función: Customer Lifetime Value
    print("\n✅ 1. Función SQL: Customer Lifetime Value")
    print("-" * 40)
    
    customer_id = 1
    months = 12
    lifetime_value = sql_demo.demo_customer_lifetime_value(customer_id, months)
    print(f"💰 Cliente {customer_id} - Valor de Vida (12 meses): ${lifetime_value:,.2f}")
    
    # Calcular para diferentes períodos
    for period in [6, 18, 24]:
        try:
            value = sql_demo.demo_customer_lifetime_value(customer_id, period)
            print(f"   Valor en {period} meses: ${value:,.2f}")
        except:
            print(f"   Valor en {period} meses: No disponible")
    
    # 2. Vista: Customer Purchase History
    print("\n✅ 2. Vista SQL: Historial de Compras de Clientes")
    print("-" * 40)
    
    purchase_history = sql_demo.demo_customer_purchase_history(customer_id)
    if purchase_history and len(purchase_history) > 0:
        print(f"📊 Registros encontrados: {len(purchase_history)}")
        if len(purchase_history) > 0:
            first_record = purchase_history[0]
            print(f"   Cliente: {first_record.get('customer_id', 'N/A')}")
            print(f"   Total gastado: ${first_record.get('total_spent', 0):,.2f}")
            print(f"   Número de compras: {first_record.get('purchase_count', 0)}")
            print(f"   Última compra: {first_record.get('last_purchase_date', 'N/A')}")
    else:
        print("   No se encontraron registros de compras")
    
    # 3. Análisis de Índices
    print("\n✅ 3. Optimización: Análisis de Uso de Índices")
    print("-" * 40)
    
    index_analysis = sql_demo.demo_index_usage()
    print("🔍 Planes de ejecución analizados:")
    
    for plan_name, plan_data in index_analysis.items():
        if plan_data and len(plan_data) > 0:
            print(f"   - {plan_name}: {len(plan_data)} pasos en el plan")
            # Mostrar información del primer paso si está disponible
            if isinstance(plan_data, list) and len(plan_data) > 0:
                first_step = plan_data[0]
                if isinstance(first_step, dict):
                    key_info = first_step.get('key', 'N/A')
                    rows_info = first_step.get('rows', 'N/A')
                    print(f"     Clave utilizada: {key_info}")
                    print(f"     Filas estimadas: {rows_info}")
        else:
            print(f"   - {plan_name}: Sin datos disponibles")
            
except Exception as e:
    print(f"❌ Error ejecutando objetos SQL: {e}")
    print("💡 Nota: Algunos objetos SQL pueden no estar creados aún en la base de datos")
    print("   Ejecuta el script sql/advanced_objects.sql para crear todos los objetos")

print(f"\n🏗️  Objetos SQL implementados:")
print("   - calculate_customer_lifetime_value(): Función para CLV")
print("   - generate_sales_report(): Procedimiento de reportes") 
print("   - after_sale_insert: Trigger de auditoría automática")
print("   - customer_purchase_history: Vista optimizada")
print("   - Índices estratégicos para performance")
print("   - sales_audit_log: Tabla de auditoría automática")


In [ ]:
# 8. Dashboard de Análisis Integral
print("\n📊 DASHBOARD EJECUTIVO - Análisis Integral de Negocio")
print("=" * 60)

try:
    # Dashboard comprensivo usando consultas avanzadas
    print("\n🎯 Generando Dashboard Ejecutivo...")
    print("-" * 40)
    
    dashboard_data = advanced_queries.advanced_sales_analytics_dashboard()
    
    if isinstance(dashboard_data, pd.DataFrame) and not dashboard_data.empty:
        print(f"📈 Dashboard generado exitosamente con {len(dashboard_data)} métricas")
        
        # Mostrar métricas clave
        print("\n📊 MÉTRICAS PRINCIPALES:")
        for i, row in dashboard_data.head(10).iterrows():
            metric_name = row.get('metric_name', f'Métrica {i+1}')
            metric_value = row.get('metric_value', 'N/A')
            category = row.get('category', 'General')
            
            if isinstance(metric_value, (int, float)):
                if 'revenue' in metric_name.lower() or 'sales' in metric_name.lower():
                    print(f"   💰 {metric_name}: ${metric_value:,.2f}")
                elif 'count' in metric_name.lower() or 'number' in metric_name.lower():
                    print(f"   📊 {metric_name}: {metric_value:,}")
                else:
                    print(f"   📈 {metric_name}: {metric_value}")
            else:
                print(f"   📋 {metric_name}: {metric_value}")
    else:
        print("   No se pudieron generar métricas del dashboard")
        
    # Análisis combinado usando múltiples patrones
    print(f"\n🔄 ANÁLISIS COMBINADO - Patrones de Diseño + SQL Avanzado")
    print("-" * 40)
    
    # Usar Strategy Pattern para análisis múltiple
    strategies = ['revenue', 'quantity', 'customer_behavior']
    context = SalesAnalysisContext()
    factory = AnalysisStrategyFactory()
    
    sample_data = [
        {'product_id': 1, 'quantity': 10, 'total_price': 100.0, 'customer_id': 1},
        {'product_id': 2, 'quantity': 15, 'total_price': 200.0, 'customer_id': 2},
    ]
    
    print("🔍 Ejecutando análisis con múltiples estrategias:")
    for strategy_name in strategies:
        try:
            strategy = factory.create_strategy(strategy_name)
            context.set_strategy(strategy)
            result = context.execute_analysis(sample_data)
            
            if strategy_name == 'revenue':
                print(f"   💰 Revenue Total: ${result:.2f}")
            elif strategy_name == 'quantity':
                print(f"   📦 Cantidad Total: {result} unidades")
            elif strategy_name == 'customer_behavior':
                print(f"   👥 Clientes Únicos: {result}")
        except Exception as e:
            print(f"   ❌ Error en estrategia {strategy_name}: {e}")
            
except Exception as e:
    print(f"❌ Error generando dashboard: {e}")

# Resumen de capacidades del sistema
print(f"\n🏆 CAPACIDADES DEL SISTEMA INTEGRADO:")
print("=" * 50)
print("✅ Patrones de Diseño Implementados:")
print("   • Singleton: Conexión única y eficiente a BD")
print("   • Factory: Creación flexible de modelos")
print("   • Builder: Construcción fluida de consultas")
print("   • Decorator: Funcionalidades transversales")
print("   • Strategy: Algoritmos de análisis intercambiables")
print()
print("✅ SQL Avanzado Implementado:")
print("   • CTEs: Consultas jerárquicas complejas")
print("   • Window Functions: Análisis temporal y ranking")
print("   • Funciones SQL: Cálculos automatizados")
print("   • Procedimientos: Reportes parametrizados")
print("   • Triggers: Auditoría automática")
print("   • Vistas Optimizadas: Consultas precalculadas")
print("   • Índices Estratégicos: Performance optimizada")
print()
print("🎯 OBJETIVOS ALCANZADOS:")
print("   • Sistema eficiente para grandes volúmenes")
print("   • Reportes en tiempo real")
print("   • Automatización de procesos clave")
print("   • Arquitectura extensible y mantenible")
print("   • Optimización de performance de consultas")
